# Decision Tree example from modul 3


## Dataset load and prepocess 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [11]:
df = pd.read_csv("500_Person_Gender_Height_Weight_Index.csv")
display(df.head())
display(df.describe())


,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3


,Height,Weight,Index
count,500.000000,500.000000,500.000000
mean,169.944000,106.000000,3.748000
std,16.375261,32.382607,1.355053
min,140.000000,50.000000,0.000000
25%,156.000000,80.000000,3.000000
50%,170.500000,106.000000,4.000000
75%,184.000000,136.000000,5.000000
max,199.000000,160.000000,5.000000


In [18]:
df['obese'] = (df.Index>=4).astype('int')
df.drop('Index',axis=1,inplace=True)
df.head()

,Gender,Height,Weight,obese
0,Male,174,96,1
1,Male,189,87,0
2,Female,185,110,1
3,Female,195,104,0
4,Male,149,61,0


In [19]:
print( 
  " Misclassified when cutting at 100kg:", 
  df.loc[(df['Weight']>=100) & (df['obese']==0),:].shape[0], "\n", 
  "Misclassified when cutting at 80kg:", 
  df.loc[(df['Weight']>=80) & (df['obese']==0),:].shape[0] 
)

 Misclassified when cutting at 100kg: 18 
 Misclassified when cutting at 80kg: 63


## Function definition

In [22]:
def entropy(y):
    if isinstance(y, pd.Series): 
        a = y.value_counts()/y.shape[0] 
        entropy = np.sum(-a*np.log2(a+1e-9)) 
        return(entropy) 
 
    else: 
        raise('Object must be a Pandas Series.')
    
entropy(df.Gender)

np.float64(0.9997114388674198)

In [25]:
def gini_impurity(y):
    if isinstance(y, pd.Series):
        p = y.value_counts()/y.shape[0]
        gini = 1-np.sum(p**2)
        return gini
    else:
        raise('Object must be a Pandas Series')

In [26]:
def variance(y):
    if(len(y) == 1): 
        return 0 
    else: 
        return y.var() 
 
def information_gain(y, mask, func=entropy): 
  ''' 
  It returns the Information Gain of a variable given a loss function. 
  y: target variable. 
  mask: split choice. 
   func:  function  to  be  used  to  calculate  Information  Gain  in  case  os 
classification. 
  ''' 
 
  a = sum(mask) 
  b = mask.shape[0] - a 
 
  if(a == 0 or b ==0): 
    ig = 0 
 
  else: 
    if y.dtypes != 'O': 
        ig  =  variance(y)  -  (a/(a+b)*  variance(y[mask]))  -  (b/(a+b)*variance(y[-
mask])) 
    else: 
      ig = func(y)-a/(a+b)*func(y[mask])-b/(a+b)*func(y[-mask]) 
 
  return ig

In [28]:
import itertools

def categorical_options(a):
    a = a.unique()
    opt = []
    for L in range(0, len(a)+1): 
      for subset in itertools.combinations(a, L): 
          subset = list(subset) 
          opt.append(subset) 
 
    return opt[1:-1]

def max_information_gain_split(x,y, func=entropy):
    split_value = [] 
    ig = [] 
 
    numeric_variable = True if x.dtypes != 'O' else False 
    # Create options according to variable type 
    if numeric_variable: 
        options = x.sort_values().unique()[1:] 
    else: 
        options = categorical_options(x) 
    
    # Calculate ig for all values 
    for val in options: 
        mask =   x < val if numeric_variable else x.isin(val) 
        val_ig = information_gain(y, mask, func) 
        # Append results 
        ig.append(val_ig) 
        split_value.append(val) 
    
    # Check if there are more than 1 results if not, return False 
    if len(ig) == 0: 
        return(None,None,None, False) 
    
    else: 
    # Get results with highest IG 
        best_ig = max(ig) 
        best_ig_index = ig.index(best_ig) 
        best_split = split_value[best_ig_index] 
        return(best_ig,best_split,numeric_variable, True) 
    
weight_ig,  weight_split,  _,  _  =  max_information_gain_split(df['Weight'], 
df['obese'],) 

print( 
"The best split for Weight is when the variable is less than ", 
weight_split,"\nInformation Gain for that split is:", weight_ig 
)



The best split for Weight is when the variable is less than  103 
Information Gain for that split is: 0.10625190497954848


In [29]:
df.drop('obese',axis=1).apply(max_information_gain_split,y=df['obese'])

,Gender,Height,Weight
0,-0.000281,0.019684,0.106252
1,[Male],174,103
2,False,True,True
3,True,True,True


In [36]:
def get_best_split(y,data):
    masks = data.drop(y, axis= 1).apply(max_information_gain_split, y = data[y]) 
    if sum(masks.loc[3,:]) == 0: 
        return(None, None, None, None) 
    
    else: 
        # Get only masks that can be splitted 
        masks = masks.loc[:,masks.loc[3,:]] 
    
        # Get the results for split with highest IG 
        split_variable = masks.iloc[0].astype(np.float32).idxmax() 
        #split_valid = masks[split_variable][] 
        split_value = masks[split_variable][1] 
        split_ig = masks[split_variable][0] 
        split_numeric = masks[split_variable][2] 
    
        return(split_variable, split_value, split_ig, split_numeric) 
 
 
def make_split(variable, value, data, is_numeric): 

  if is_numeric: 
    data_1 = data[data[variable] < value] 
    data_2 = data[(data[variable] < value) == False] 
 
  else: 
    data_1 = data[data[variable].isin(value)] 
    data_2 = data[(data[variable].isin(value)) == False]

  return (data_1,data_2)
  

def make_predictions(data, target_factor):
    if target_factor:
        pred = data.value_counts().idxmax()
    else:
        pred = data.mean()

    return pred



In [38]:
def train_tree(data, y, target_factor, max_depth=None, min_samples_split=None, min_information_gain=1e-20,counter=0, max_categories = 20):
         
    # Trains a Decission Tree 
    # data: Data to be used to train the Decission Tree 
    # y: target variable column name 
    # target_factor: boolean to consider if target variable is factor or numeric. 
    # max_depth: maximum depth to stop splitting. 
    # min_samples_split: minimum number of observations to make a split. 
    # min_information_gain: minimum ig gain to consider a split to be valid. 
    # max_categories:  maximum  number  of  different  values  accepted  for  categorical 
    # values. High number of values will slow down learning process. R 

# Check that max_categories is fulfilled 
  if counter==0: 
    types = data.dtypes 
    check_columns = types[types == "object"].index

    for column in check_columns:
      var_length = len(data[column].value_counts()) 
      if var_length > max_categories: 
        raise ValueError('The variable ' + column + ' has '+ str(var_length) + ' unique values, which is more than the accepted ones: ' +  str(max_categories)) 
 
  # Check for depth conditions 
  if max_depth == None: 
    depth_cond = True 
 
  else: 
    if counter < max_depth: 
      depth_cond = True 
 
    else: 
      depth_cond = False 
 
  # Check for sample conditions 
  if min_samples_split == None: 
    sample_cond = True 
 
  else: 
    if data.shape[0] > min_samples_split: 
      sample_cond = True 
 
    else: 
      sample_cond = False 
 
  # Check for ig condition 
  if depth_cond & sample_cond: 
 
    var,val,ig,var_type = get_best_split(y, data) 
 
    # If ig condition is fulfilled, make split 
    if ig is not None and ig >= min_information_gain: 
 
      counter += 1 
 
      left,right = make_split(var, val, data,var_type) 
 
      # Instantiate sub-tree 
      split_type = "<=" if var_type else "in" 
      question =   "{} {}  {}".format(var,split_type,val) 
#       question = "\n" + counter*" " + "|->" + var + " " + split_type + " " + 
# str(val) 
      subtree = {question: []} 
 
      # Find answers (recursion) 
      yes_answer = train_tree(left,y, target_factor, max_depth,min_samples_split,min_information_gain, counter) 
 
      no_answer = train_tree(right,y, target_factor, max_depth,min_samples_split,min_information_gain, counter) 
 
      if yes_answer == no_answer: 
        subtree = yes_answer 
      else: 
        subtree[question].append(yes_answer) 
        subtree[question].append(no_answer) 
    
        # If it doesn't match IG condition, make prediction 
    else: 
     pred = make_predictions(data[y],target_factor) 
     return pred 
    
    # Drop dataset if doesn't match depth or sample conditions 

    
    return subtree 
 
max_depth = 5 
min_samples_split = 20 
min_information_gain  = 1e-5 
 
decision = train_tree(df,'obese',True, max_depth,min_samples_split,min_information_gain) 
decision 
    
    
    


{'Weight <=  103': [{'Height <=  175': [{'Weight <=  74': [None,
      {'Height <=  162': [np.int64(1), None]}]},
    {'Height <=  178': [None, np.int64(0)]}]},
  {'Height <=  189': [{'Weight <=  116': [{'Height <=  168': [np.int64(1),
        None]},
      np.int64(1)]},
    {'Weight <=  115': [None, np.int64(1)]}]}]}